
# SQL ➜ Python/SQLite: do script ao dashboard (Colab/Notebook)

Este notebook carrega e executa o arquivo **`scripts_atividade.sql`** (DDL + DML),
cria um banco **SQLite** local, faz consultas com **pandas**, trata nulos e gera
uma visualização simples. Ideal para demonstrar um fluxo básico em aulas.

> **Arquivos usados**
> - SQL: `/mnt/data/scripts_atividade.sql`
> - DB SQLite (gerado): `/mnt/data/loja.db`
> - Saídas (CSV): criadas em `/mnt/data/` ao final.


In [ ]:

# %% [markdown]
# ## 1) Setup

import sqlite3, pandas as pd, numpy as np, matplotlib.pyplot as plt, textwrap, os, sys
from pathlib import Path

print('Python', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('matplotlib', plt.matplotlib.__version__)

DB_PATH = Path('/mnt/data/loja.db')
SQL_PATH = Path('/mnt/data/scripts_atividade.sql')
DB_PATH.unlink(missing_ok=True)  # começa limpo


In [ ]:

# %% [markdown]
# ## 2) Carregar e executar o script SQL

assert SQL_PATH.exists(), f'Arquivo SQL não encontrado: {SQL_PATH}'

with open(SQL_PATH, 'r', encoding='utf-8') as f:
    sql_script = f.read()

# Cria conexão e executa o script completo (DDL + inserts)
con = sqlite3.connect(DB_PATH)
cur = con.cursor()
cur.executescript(sql_script)
con.commit()

# Verificação simples: listar as tabelas criadas
tabelas = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", con)
tabelas


In [ ]:

# %% [markdown]
# ## 3) Visão rápida das tabelas

df_produtos = pd.read_sql_query('SELECT * FROM produtos;', con)
df_vendas   = pd.read_sql_query('SELECT * FROM vendas;', con)

display(df_produtos.head(10))
display(df_vendas.head(10))

print('\nDimensões:')
print('produtos:', df_produtos.shape)
print('vendas  :', df_vendas.shape)


In [ ]:

# %% [markdown]
# ## 4) Diagnóstico de nulos e inconsistências

def resumo_nulos(df, nome):
    nulos = df.isna().sum().rename('n_nulos')
    tipos = df.dtypes.rename('dtype')
    out = pd.concat([tipos, nulos], axis=1)
    out.index.name = nome
    return out

print('--- Nulos em PRODUTOS ---')
display(resumo_nulos(df_produtos, 'produtos'))
print('\n--- Nulos em VENDAS ---')
display(resumo_nulos(df_vendas, 'vendas'))

# Casos específicos observados no script:
# - produtos.preco_unitario pode ser NULL (ex.: 'Monitor Samsung 27"')
# - produtos.categoria pode ser NULL (ex.: 'Caixa de Som JBL')
# - vendas.quantidade pode ser NULL
# - vendas.desconto pode ser NULL
# - vendas.id_produto pode ser NULL (linha de venda sem produto)

# Contagens úteis:
print('\nLinhas de vendas com id_produto NULL:', df_vendas['id_produto'].isna().sum())
print('Linhas de vendas com quantidade NULL  :', df_vendas['quantidade'].isna().sum())
print('Linhas de vendas com desconto NULL    :', df_vendas['desconto'].isna().sum())
print('Produtos com preco_unitario NULL      :', df_produtos['preco_unitario'].isna().sum())
print('Produtos com categoria NULL           :', df_produtos['categoria'].isna().sum())


In [ ]:

# %% [markdown]
# ## 5) Join e métricas de receita (tratando nulos)

q = '''
SELECT
  v.id_venda,
  v.id_produto,
  p.nome_produto,
  p.categoria,
  p.preco_unitario,
  v.quantidade,
  v.data_venda,
  v.desconto
FROM vendas v
LEFT JOIN produtos p ON p.id_produto = v.id_produto
ORDER BY v.id_venda;
'''
df_join = pd.read_sql_query(q, con)

df_join['qtd_is_null']  = df_join['quantidade'].isna()
df_join['desc_is_null'] = df_join['desconto'].isna()
df_join['preco_is_null']= df_join['preco_unitario'].isna()

df_join['quantidade_filled']  = df_join['quantidade'].fillna(0)
df_join['desconto_filled']    = df_join['desconto'].fillna(0.0)
df_join['preco_filled']       = df_join['preco_unitario'].fillna(0.0)

df_join['receita_bruta'] = df_join['preco_filled'] * df_join['quantidade_filled']
df_join['receita_liq']   = df_join['receita_bruta'] - df_join['desconto_filled']

display(df_join.head(12))

from pathlib import Path
out_join_csv = Path('/mnt/data/vendas_join_metricas.csv')
df_join.to_csv(out_join_csv, index=False)
print('CSV salvo:', out_join_csv)


In [ ]:

# %% [markdown]
# ## 6) Agregações úteis

cat_agg = (df_join
           .groupby('categoria', dropna=False, as_index=False)
           [['receita_bruta','receita_liq']].sum()
           .sort_values('receita_liq', ascending=False))
display(cat_agg)

prod_agg = (df_join
            .groupby(['id_produto','nome_produto'], dropna=False, as_index=False)
            [['receita_bruta','receita_liq']].sum()
            .sort_values('receita_liq', ascending=False))
display(prod_agg.head(15))

from pathlib import Path
cat_csv  = Path('/mnt/data/receita_por_categoria.csv')
prod_csv = Path('/mnt/data/receita_por_produto.csv')
cat_agg.to_csv(cat_csv, index=False)
prod_agg.to_csv(prod_csv, index=False)
print('CSVs salvos:', cat_csv, '|', prod_csv)


In [ ]:

# %% [markdown]
# ## 7) Visualização (matplotlib, 1 gráfico, sem cores definidas)

import matplotlib.pyplot as plt

plt.figure()
vals = cat_agg.fillna({'categoria':'(Sem categoria)'})
plt.bar(vals['categoria'], vals['receita_liq'])
plt.title('Receita líquida por categoria')
plt.xlabel('Categoria')
plt.ylabel('Receita líquida (R$)')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()



## 8) Dicas rápidas vistas em aula

- **`executemany`** para inserir em lote a partir de listas.
- **Transações**: `INSERT/UPDATE/DELETE` ficam pendentes até `conn.commit()`; é possível reverter com **rollback** antes do commit.
- **Arquivos `.txt/.csv`**: leia linha a linha, faça `split(',')`, valide o número de colunas e insira.
- **`DELETE FROM tabela`** remove todos os registros; cuidado (prefira `WHERE`).
- **Chaves** e **restrições**: avalie `PRIMARY KEY`, `NOT NULL`, `UNIQUE` de acordo com o modelo de dados.
- **LEFT JOIN** ajuda a diagnosticar problemas de integridade referencial (ex.: `id_produto` ausente em `vendas`).

> Siga os mesmos conceitos no VS Code, Jupyter local ou Google Colab; apenas ajuste o caminho do banco/arquivos.



---

# 9) SELECT com mais recursos (continuação do curso básico)

Nesta seção vamos trabalhar com uma tabela didática chamada **`inscricoes`** para praticar
**seleção**, **filtros com `WHERE`**, conectores **`AND`/`OR`**, **aliases (`AS`)** e leitura de resultados com
**`fetchall`**, **`fetchone`** e **`fetchmany`**.


In [ ]:

# %% [markdown]
# ### 9.1) Criar a tabela `inscricoes` (se não existir) e inserir poucos dados

import sqlite3
from pathlib import Path

DB_PATH = Path('/mnt/data/loja.db')

# Tenta reutilizar a conexão `con` criada antes; se não existir, abre novamente.
try:
    con
except NameError:
    con = sqlite3.connect(DB_PATH)
cur = con.cursor()

cur.executescript('''
CREATE TABLE IF NOT EXISTS inscricoes (
    nome   TEXT NOT NULL,
    email  TEXT NOT NULL,
    curso  TEXT NOT NULL,
    idade  INTEGER
);

DELETE FROM inscricoes;  -- mantemos a seção reprodutível

INSERT INTO inscricoes (nome, email, curso, idade) VALUES
  ('Ivan',   'ivan@gmail.com',    'Java',    50),
  ('Denise', 'denise@gmail.com',  'Java',    38),
  ('Joana',  'joana@gmail.com',   'Java',    28),
  ('Carlos', 'carlos@gmail.com',  'Dados',   35),
  ('Silvio', 'silvio@gmail.com',  'Python',  67);
''')
con.commit()

import pandas as pd
df_ins = pd.read_sql_query('SELECT * FROM inscricoes;', con)
df_ins


In [ ]:

# %% [markdown]
# ### 9.2) Selecionando colunas específicas e usando alias (`AS`)

import pandas as pd
q_alias = '''
SELECT
  nome,
  email,
  curso AS tipo  -- alias renomeia a coluna no resultado
FROM inscricoes;
'''
pd.read_sql_query(q_alias, con)


In [ ]:

# %% [markdown]
# ### 9.3) `WHERE`: filtrando somente quem faz **Java**

q_java = '''
SELECT nome, email, curso, idade
FROM inscricoes
WHERE curso = 'Java';
'''
pd.read_sql_query(q_java, con)


In [ ]:

# %% [markdown]
# ### 9.4) `AND` vs `OR`: combinando condições
# - **AND**: precisa satisfazer **todas** as condições (interseção)
# - **OR** : satisfazer **pelo menos uma** (união)

# AND: curso = 'Java' **e** idade >= 40
q_and = '''
SELECT nome, email, curso, idade
FROM inscricoes
WHERE curso = 'Java' AND idade >= 40;
'''
print("AND (Java **e** idade >= 40)")
display(pd.read_sql_query(q_and, con))

# OR: curso = 'Java' **ou** idade >= 40
q_or = '''
SELECT nome, email, curso, idade
FROM inscricoes
WHERE curso = 'Java' OR idade >= 40;
'''
print("\nOR (Java **ou** idade >= 40)")
display(pd.read_sql_query(q_or, con))


In [ ]:

# %% [markdown]
# ### 9.5) Lendo resultados com `fetchall`, `fetchone` e `fetchmany`

import sqlite3

c = con.cursor()
c.execute("SELECT nome, email, curso, idade FROM inscricoes ORDER BY nome;")

print("-> fetchall(): retorna **todas** as linhas de uma vez")
c.execute("SELECT nome, email, curso, idade FROM inscricoes ORDER BY nome;")
dados_all = c.fetchall()
print(f"linhas: {len(dados_all)} | primeiras 2:", dados_all[:2])

print("\n-> fetchone(): retorna **uma** linha por chamada (cursor avança a cada chamada)")
c.execute("SELECT nome, email, curso, idade FROM inscricoes ORDER BY nome;")
um1 = c.fetchone()
um2 = c.fetchone()
print("primeira:", um1)
print("segunda :", um2)

print("\n-> fetchmany(n): retorna **n** linhas por chamada")
c.execute("SELECT nome, email, curso, idade FROM inscricoes ORDER BY nome;")
lote1 = c.fetchmany(2)
lote2 = c.fetchmany(2)
lote3 = c.fetchmany(2)  # pode vir vazio se terminar
print("lote1 (2):", lote1)
print("lote2 (2):", lote2)
print("lote3 (2):", lote3)



> **Dica**: Combine `fetchmany(n)` com `ORDER BY` para processar **lotes** previsíveis
sem carregar tudo na memória de uma vez. Se você precisa de *página 1, 2, 3...*,
também pode usar **`LIMIT`/`OFFSET`** diretamente no SQL.



---

# 10) Relatório com **string formatada** (tabular, pronto para impressão)

Nesta parte vamos gerar um relatório tabular usando **formatação de strings** do Python
(placeholder com larguras fixas), simulando uma saída pronta para impressão/PDF.
Trabalharemos sobre a tabela **`inscricoes`** criada anteriormente.


In [ ]:

# %% [markdown]
# ### 10.1) Funções auxiliares para relatório tabular

import sqlite3, pandas as pd
from pathlib import Path

DB_PATH = Path('/mnt/data/loja.db')
try:
    con
except NameError:
    con = sqlite3.connect(DB_PATH)

# Garante a tabela didática `inscricoes`
con.execute('''
CREATE TABLE IF NOT EXISTS inscricoes (
    nome   TEXT NOT NULL,
    email  TEXT NOT NULL,
    curso  TEXT NOT NULL,
    idade  INTEGER
);
''')
con.commit()

# Larguras de coluna (Nome, Email, Curso, Idade)
W_NOME, W_EMAIL, W_CURSO, W_IDADE = 30, 30, 15, 5

def imprimir_cabecalho(titulo="Consulta de inscritos", subtitulo="Dados da tabela inscricoes"):
    print("\n" + titulo)
    print(subtitulo)
    print("-" * 80)
    print("{:<{w1}}{:<{w2}}{:<{w3}}{:>{w4}}".format("Nome", "E-mail", "Curso", "Idade",
                                                    w1=W_NOME, w2=W_EMAIL, w3=W_CURSO, w4=W_IDADE))
    print("=" * 80)

def imprimir_linhas(linhas):
    for linha in linhas:
        nome, email, curso, idade = linha
        print("{:<{w1}}{:<{w2}}{:<{w3}}{:>{w4}}".format(
            str(nome)[:W_NOME],
            str(email)[:W_EMAIL],
            str(curso)[:W_CURSO],
            "" if idade is None else f"{idade:d}",
            w1=W_NOME, w2=W_EMAIL, w3=W_CURSO, w4=W_IDADE
        ))

def imprimir_rodape(total):
    print("-" * 80)
    print("Foram encontrados {} registros.".format(total))
    print("\n")


In [ ]:

# %% [markdown]
# ### 10.2) Relatório completo (usando `fetchall`) – string formatada

cur = con.cursor()
cur.execute("SELECT nome, email, curso, idade FROM inscricoes ORDER BY nome;")
dados = cur.fetchall()  # lista de tuplas

imprimir_cabecalho()
imprimir_linhas(dados)
imprimir_rodape(len(dados))


In [ ]:

# %% [markdown]
# ### 10.3) Paginação com `fetchmany(n)` – útil para "páginas" de 20 registros

PAGE_SIZE = 20

cur = con.cursor()
cur.execute("SELECT nome, email, curso, idade FROM inscricoes ORDER BY nome;")

pagina = 1
while True:
    bloco = cur.fetchmany(PAGE_SIZE)  # lê próximo lote
    if not bloco:
        break
    imprimir_cabecalho(titulo=f"Consulta de inscritos (página {pagina})")
    imprimir_linhas(bloco)
    imprimir_rodape(total=len(bloco))
    pagina += 1


In [ ]:

# %% [markdown]
# ### 10.4) (Opcional) Aumentar massa de dados para testar paginação
# Execute esta célula se quiser duplicar entradas e ver múltiplas páginas.

dados_extra = [
    ('Ana',    'ana@example.com',    'Java',   22),
    ('Beto',   'beto@example.com',   'Python', 41),
    ('Clara',  'clara@example.com',  'Dados',  29),
    ('Davi',   'davi@example.com',   'Java',   35),
    ('Eva',    'eva@example.com',    'Python', 48),
]

# Insere os mesmos 5 registros várias vezes (ajuste N para variar o volume)
N = 4  # 5*N novas linhas
cur = con.cursor()
for _ in range(N):
    cur.executemany("INSERT INTO inscricoes (nome, email, curso, idade) VALUES (?, ?, ?, ?);", dados_extra)
con.commit()

# Mostra uma prévia
pd.read_sql_query("SELECT COUNT(*) AS total FROM inscricoes;", con)


In [ ]:

# %% [markdown]
# ### 10.5) Relatório paginado novamente após inserir dados extra

PAGE_SIZE = 20

cur = con.cursor()
cur.execute("SELECT nome, email, curso, idade FROM inscricoes ORDER BY nome;")

pagina = 1
while True:
    bloco = cur.fetchmany(PAGE_SIZE)
    if not bloco:
        break
    imprimir_cabecalho(titulo=f"Consulta de inscritos (página {pagina})")
    imprimir_linhas(bloco)
    imprimir_rodape(total=len(bloco))
    pagina += 1



---

# 11) Exportar relatório para arquivos (TXT e CSV)

Nesta seção salvamos o relatório tabular em disco (**.txt**) e também exportamos os dados em **.csv**,
mantendo a mesma ordenação. A paginação é opcional.


In [ ]:

# %% [markdown]
# ### 11.1) Gerar e salvar relatório TXT (com paginação)

import sqlite3, csv
from pathlib import Path

DB_PATH = Path('/mnt/data/loja.db')
TXT_PATH = Path('/mnt/data/relatorio_inscricoes.txt')
CSV_PATH = Path('/mnt/data/relatorio_inscricoes.csv')

try:
    con
except NameError:
    con = sqlite3.connect(DB_PATH)

cur = con.cursor()
cur.execute("SELECT nome, email, curso, idade FROM inscricoes ORDER BY nome;")

W_NOME, W_EMAIL, W_CURSO, W_IDADE = 30, 30, 15, 5
PAGE_SIZE = 40  # altere se quiser

def header_txt(page):
    return (
        "\nConsulta de inscritos (página {})\n".format(page) +
        "Dados da tabela inscricoes\n" +
        "-"*80 + "\n" +
        "{:<{w1}}{:<{w2}}{:<{w3}}{:>{w4}}\n".format("Nome","E-mail","Curso","Idade",
                                                     w1=W_NOME, w2=W_EMAIL, w3=W_CURSO, w4=W_IDADE) +
        "="*80 + "\n"
    )

with TXT_PATH.open('w', encoding='utf-8') as f:
    page = 1
    count_in_page = 0
    f.write(header_txt(page))
    total = 0

    while True:
        bloco = cur.fetchmany(PAGE_SIZE)
        if not bloco:
            break
        for nome, email, curso, idade in bloco:
            f.write("{:<{w1}}{:<{w2}}{:<{w3}}{:>{w4}}\n".format(
                str(nome)[:W_NOME], str(email)[:W_EMAIL], str(curso)[:W_CURSO],
                "" if idade is None else f"{idade:d}",
                w1=W_NOME, w2=W_EMAIL, w3=W_CURSO, w4=W_IDADE
            ))
            total += 1
            count_in_page += 1
            if count_in_page >= PAGE_SIZE:
                f.write("-"*80 + "\n")
                f.write("Página {} — {} registros nesta página\n".format(page, count_in_page))
                f.write("\f")  # form feed (quebra lógica de página)
                page += 1
                count_in_page = 0
                f.write(header_txt(page))

    # Rodapé final
    f.write("-"*80 + "\n")
    f.write("Total geral: {} registros\n".format(total))

TXT_PATH, TXT_PATH.exists()

In [ ]:

# %% [markdown]
# ### 11.2) Exportar os mesmos dados para CSV

import pandas as pd

df_csv = pd.read_sql_query("SELECT nome, email, curso, idade FROM inscricoes ORDER BY nome;", con)
CSV_PATH = Path('/mnt/data/relatorio_inscricoes.csv')
df_csv.to_csv(CSV_PATH, index=False)
CSV_PATH, CSV_PATH.exists(), df_csv.head()


---

# 12) Relatório **agrupado por curso** com totalizadores

Vamos criar um resumo por **curso**, com **contagem de inscritos** e **idade média**.
Além do DataFrame, geramos também uma saída **formatada** em texto.


In [ ]:

# %% [markdown]
# ### 12.1) Agregações (COUNT, AVG) e ordenação

import pandas as pd
import numpy as np

q_group = '''
SELECT
  curso,
  COUNT(*)                AS total_inscritos,
  AVG(COALESCE(idade, 0)) AS idade_media  -- usa 0 quando NULL para não perder linhas
FROM inscricoes
GROUP BY curso
ORDER BY total_inscritos DESC, curso;
'''
df_group = pd.read_sql_query(q_group, con)
df_group['idade_media'] = df_group['idade_media'].round(1)
df_group


In [ ]:

# %% [markdown]
# ### 12.2) Saída agrupada formatada (texto)

TXT_GROUP_PATH = Path('/mnt/data/relatorio_por_curso.txt')

W_CURSO, W_TOTAL, W_MEDIA = 15, 8, 12
with TXT_GROUP_PATH.open('w', encoding='utf-8') as f:
    f.write("\nResumo por curso\n")
    f.write("-"*45 + "\n")
    f.write("{:<{w1}}{:>{w2}}{:>{w3}}\n".format("Curso","Inscritos","Idade média",
                                                w1=W_CURSO, w2=W_TOTAL, w3=W_MEDIA))
    f.write("="*45 + "\n")
    for _, row in df_group.iterrows():
        f.write("{:<{w1}}{:>{w2}d}{:>{w3}.1f}\n".format(
            str(row['curso'])[:W_CURSO], int(row['total_inscritos']), float(row['idade_media']),
            w1=W_CURSO, w2=W_TOTAL, w3=W_MEDIA
        ))
    f.write("-"*45 + "\n")
    f.write("Total de cursos: {}\n".format(len(df_group)))

TXT_GROUP_PATH, TXT_GROUP_PATH.exists()


---

# 13) Manipulação de dados (**DELETE** & **UPDATE**)

Nesta aula vamos praticar **deleção** e **alteração** de registros na tabela didática `inscricoes`.

**Roteiro:**  
1. Apagar tudo da tabela (`DELETE FROM`).  
2. Repopular com ~100 linhas a partir de um arquivo **TXT** (gerado automaticamente se não existir).  
3. Contagens com `COUNT` e sumarização com `GROUP BY` + `ORDER BY` (usando *alias*).  
4. `DELETE` condicional (ex.: remover quem é `Excel`).  
5. `UPDATE` de um único registro (por e-mail) e `UPDATE` em lote (todos de Java -10 anos).


In [ ]:

# %% [markdown]
# ### 13.1) Apagar tudo da tabela `inscricoes` e verificar

import sqlite3, pandas as pd
from pathlib import Path

DB_PATH = Path('/mnt/data/loja.db')
try:
    con
except NameError:
    con = sqlite3.connect(DB_PATH)

con.execute('''
CREATE TABLE IF NOT EXISTS inscricoes (
    nome   TEXT NOT NULL,
    email  TEXT NOT NULL,
    curso  TEXT NOT NULL,
    idade  INTEGER
);
''')
con.commit()

con.execute("DELETE FROM inscricoes;")
con.commit()

pd.read_sql_query("SELECT COUNT(*) AS totalapos_delete FROM inscricoes;", con)

In [ ]:

# %% [markdown]
# ### 13.2) Inserir 100 linhas a partir de TXT (gera se não existir)

import random, csv, os

TXT_DADOS = Path('/mnt/data/dados_aleatorios.txt')

def gerar_txt_dados(path: Path, n=100):
    random.seed(42)
    nomes = ["Ana","Bruno","Carla","Davi","Eduardo","Fernanda","Gabriela","Heitor","Isabela","João","Karina","Luiz","Mariana","Natália","Otávio","Paula","Quésia","Rafaela","Sergio","Talita","Ursula","Valter","Wagner","Xênia","Yuri","Zilda"]
    sobrenomes = ["Silva","Santos","Oliveira","Souza","Costa","Pereira","Rodrigues","Almeida","Gomes","Barbosa","Ribeiro","Carvalho","Araujo","Fernandes","Rocha","Dias","Moreira","Nunes","Machado","Lima","Freitas","Melo"]
    dominios = ["example.com","mail.com","gmail.com","live.com","outlook.com"]
    cursos = ["Python","Java","Dados","BI","Excel"]
    idades = list(range(18, 91))

    with path.open('w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # OBS: sem cabeçalho, conforme o fluxo da aula
        for _ in range(n):
            nome = f"{random.choice(nomes)} {random.choice(sobrenomes)}"
            email = f"{nome.lower().replace(' ','.')}@{random.choice(dominios)}"
            curso = random.choice(cursos)
            idade = random.choice(idades)
            writer.writerow([nome, email, curso, idade])

# Gera se não existir
if not TXT_DADOS.exists():
    gerar_txt_dados(TXT_DADOS, n=100)

# Inserção a partir do TXT
cur = con.cursor()
with TXT_DADOS.open('r', encoding='utf-8') as f:
    reader = csv.reader(f)
    lote = []
    for row in reader:
        if not row or len(row) < 4:
            continue
        nome, email, curso, idade = row[0], row[1], row[2], int(row[3])
        lote.append((nome, email, curso, idade))
    cur.executemany("INSERT INTO inscricoes (nome, email, curso, idade) VALUES (?, ?, ?, ?);", lote)
con.commit()

# Ver a amostra
df_ins = pd.read_sql_query("SELECT * FROM inscricoes LIMIT 5;", con)
display(df_ins)
print("Total atual em inscricoes:", pd.read_sql_query("SELECT COUNT(*) AS total FROM inscricoes;", con).iloc[0,0])
print("TXT salvo em:", TXT_DADOS)

In [ ]:

# %% [markdown]
# ### 13.3) Contagens e sumarização (COUNT, GROUP BY, ORDER BY com alias)

# Count simples por filtro
q_java = "SELECT COUNT(*) AS total_java FROM inscricoes WHERE curso = 'Java';"
q_python = "SELECT COUNT(*) AS total_python FROM inscricoes WHERE curso = 'Python';"
display(pd.read_sql_query(q_java, con))
display(pd.read_sql_query(q_python, con))

# Frequência por curso com alias e ordenação por total (decrescente)
q_group = '''
SELECT
  curso                                  AS tipo,
  COUNT(*)                                AS total
FROM inscricoes
GROUP BY curso
ORDER BY total DESC, tipo;
'''
df_freq = pd.read_sql_query(q_group, con)
df_freq

In [ ]:

# %% [markdown]
# ### 13.4) `DELETE` condicional: remover todos com `curso = 'Excel'`

print("Antes (Excel):", pd.read_sql_query("SELECT COUNT(*) AS n FROM inscricoes WHERE curso='Excel';", con).iloc[0,0])
con.execute("DELETE FROM inscricoes WHERE curso = 'Excel';")
con.commit()
print("Depois (Excel):", pd.read_sql_query("SELECT COUNT(*) AS n FROM inscricoes WHERE curso='Excel';", con).iloc[0,0])
print("Total atual:", pd.read_sql_query("SELECT COUNT(*) AS total FROM inscricoes;", con).iloc[0,0])

In [ ]:

# %% [markdown]
# ### 13.5) `UPDATE` individual (por e-mail) e `UPDATE` em lote (todos de Java -10 anos)

import pandas as pd

# Escolhe 1 pessoa de Java aleatoriamente para demonstrar UPDATE individual
row = pd.read_sql_query("SELECT email, nome, idade FROM inscricoes WHERE curso='Java' ORDER BY RANDOM() LIMIT 1;", con)
if not row.empty:
    email_escolhido = row.loc[0,'email']
    idade_antes = int(row.loc[0,'idade'])
    con.execute("UPDATE inscricoes SET idade = ? WHERE email = ?;", (idade_antes + 5, email_escolhido))
    con.commit()
    idade_depois = pd.read_sql_query("SELECT idade FROM inscricoes WHERE email = ?;", con, params=(email_escolhido,)).iloc[0,0]
    print(f"UPDATE individual -> {email_escolhido}: {idade_antes} -> {idade_depois}")
else:
    print("Não há registros de Java para demonstrar UPDATE individual.")

# UPDATE em lote: todos os Java rejuvenescem 10 anos (apenas demonstração)
con.execute("UPDATE inscricoes SET idade = idade - 10 WHERE curso = 'Java';")
con.commit()

# Checagem rápida
df_check = pd.read_sql_query("SELECT curso, COUNT(*) AS n, ROUND(AVG(idade),1) AS idade_media FROM inscricoes GROUP BY curso ORDER BY curso;", con)
df_check